In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_MA.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_MA.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_MA_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__MA_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_MA.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_MA_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_MA.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("East_Massachusetts_Apr_weekend");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_traffic_assignment_uni-class/East_Massachusetts_trips_Apr_weekend.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 64 entries:
  (3,6) => 492.255174
  (4,4) => 0.0
  (3,1) => 550.454014
  (4,5) => 297.8199
  (2,4) => 262.686455
  (6,5) => 337.64275
  (8,4) => 393.749843
  (1,2) => 469.814113
  (5,1) => 357.213346
  (3,4) => 337.58996
  (8,2) => 204.437727
  (2,3) => 370.060645
  (2,6) => 435.446049
  (5,4) => 315.592454
  (2,1) => 497.168947
  (2,5) => 416.028957
  (8,5) => 302.149252
  (1,4) => 216.960225
  (2,8) => 280.070247
  (5,8) => 277.481654
  (4,8) => 207.332502
  (7,8) => 272.515276
  (1,1) => 0.0
  (7,7) => 0.0
  (5,7) => 486.898716
  ⋮     => ⋮

In [7]:
odPairLabel_

Dict{UTF8String,Any} with 56 entries:
  "32" => {5,4}
  "29" => {5,1}
  "1"  => {1,2}
  "54" => {8,5}
  "2"  => {1,3}
  "41" => {6,7}
  "53" => {8,4}
  "51" => {8,2}
  "27" => {4,7}
  "42" => {6,8}
  "33" => {5,6}
  "28" => {4,8}
  "52" => {8,3}
  "50" => {8,1}
  "26" => {4,6}
  "10" => {2,4}
  "24" => {4,3}
  "25" => {4,5}
  "23" => {4,2}
  "49" => {7,8}
  "5"  => {1,6}
  "31" => {5,3}
  "43" => {7,1}
  "39" => {6,4}
  "44" => {7,2}
  ⋮    => ⋮

In [8]:
demandsVec = zeros(56)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

56-element Array{Float64,1}:
 469.814
 673.788
 216.96 
 419.865
 320.701
 388.629
 229.399
 497.169
 370.061
 262.686
 416.029
 435.446
 367.146
   ⋮    
 334.435
 332.73 
 339.458
 287.751
 272.515
 238.571
 204.438
 337.006
 393.75 
 302.149
 325.533
 389.839

In [10]:
for key=keys(odPairRoute)
    if contains(key, "56-")
        println(key)
    end
end

56-314
56-313
56-312


In [11]:
linkRoute

Dict{UTF8String,Any} with 986 entries:
  "5-260"  => 1.0
  "3-290"  => 1.0
  "20-207" => 1.0
  "16-217" => 1.0
  "6-198"  => 1.0
  "19-180" => 1.0
  "23-180" => 1.0
  "16-48"  => 1.0
  "10-128" => 1.0
  "16-266" => 1.0
  "11-148" => 1.0
  "14-299" => 1.0
  "21-112" => 1.0
  "18-290" => 1.0
  "16-235" => 1.0
  "5-70"   => 1.0
  "9-93"   => 1.0
  "9-60"   => 1.0
  "1-14"   => 1.0
  "11-19"  => 1.0
  "17-190" => 1.0
  "11-52"  => 1.0
  "19-65"  => 1.0
  "10-291" => 1.0
  "12-241" => 1.0
  ⋮        => ⋮

In [12]:
coeffs_dict = readall("../temp_files/coeffs_dict_Apr_weekend.json")
coeffs_dict = JSON.parse(coeffs_dict)
fcoeffs = coeffs_dict["(8,0.5,10000.0,1)"]

9-element Array{Any,1}:
  1.0       
 -0.00285055
  0.0535364 
 -0.209553  
  0.582268  
 -0.92582   
  0.892288  
 -0.468562  
  0.103891  

In [13]:
using JuMP

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 24
numRoute = 314
numOD = 56

@defVar(m, linkFlow[1:numLinks])
@defVar(m, pathFlow[1:numRoute])

pathFlowSum = Dict()

for i=1:numOD
    pathFlowSum[i] = 0
    for j=1:numRoute
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:numLinks
    pathFlowLinkSum[a] = 0
    for j=1:numRoute
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:numRoute
    @addConstraint(m, pathFlow[j] >= 0)
end

# @defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:numLinks})

@defNLExpr(f, sum{free_flow_time[a] * fcoeffs[1] * linkFlow[a] + 
    free_flow_time[a] * fcoeffs[2] * linkFlow[a]^2 / capacity[a] +
    free_flow_time[a] * fcoeffs[3] * linkFlow[a]^3 / capacity[a]^2 +
    free_flow_time[a] * fcoeffs[4] * linkFlow[a]^4 / capacity[a]^3 +
    free_flow_time[a] * fcoeffs[5] * linkFlow[a]^5 / capacity[a]^4 +
    free_flow_time[a] * fcoeffs[6] * linkFlow[a]^6 / capacity[a]^5 +
    free_flow_time[a] * fcoeffs[7] * linkFlow[a]^7 / capacity[a]^6 +
    free_flow_time[a] * fcoeffs[8] * linkFlow[a]^8 / capacity[a]^7 +
    free_flow_time[a] * fcoeffs[9] * linkFlow[a]^9 / capacity[a]^8, a = 1:numLinks})

@setNLObjective(m, Min, f)

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1324
Number of nonzeros in inequality constraint Jacobian.:      314
Number of nonzeros in Lagrangian Hessian.............:       24

Total number of variables............................:      338
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [14]:
getValue(linkFlow)

24-element Array{Float64,1}:
 2489.76 
  710.807
  229.4  
 1885.53 
 3671.74 
 1761.14 
  479.647
  396.9  
 3555.53 
 3054.14 
 1248.4  
 1222.27 
 1167.27 
 1268.17 
  690.164
  619.896
  207.333
  393.75 
 2949.78 
 2791.24 
  539.065
  613.285
 1638.47 
 1797.54 

In [15]:
getObjectiveValue(m)

8101.95259236019

In [16]:
flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = getValue(linkFlow)[i]
end

flows

Dict{(Int64,Int64),Float64} with 24 entries:
  (3,6) => 1248.4021706562737
  (2,3) => 3671.736039557336
  (5,6) => 690.1643740868645
  (5,4) => 1268.1657069498056
  (2,1) => 710.806532209143
  (6,3) => 1222.2650027062286
  (3,1) => 1885.5320036167768
  (1,3) => 229.39950341135184
  (4,5) => 1167.2678959567706
  (4,8) => 207.332501915158
  (7,5) => 2791.243461299993
  (8,7) => 1797.535590932901
  (3,2) => 1761.1353313459633
  (7,6) => 613.2845665662535
  (2,4) => 479.6466797438978
  (7,8) => 1638.4675969458124
  (6,5) => 619.8959610864896
  (6,7) => 539.0648805166733
  (8,4) => 393.74984292807045
  (1,2) => 2489.758223414568
  (5,3) => 3054.1396602714526
  (4,2) => 396.90038474984505
  (3,5) => 3555.534735327354
  (5,7) => 2949.778350362485

In [17]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekend

No dicts found; please check load_dicts...


In [18]:
flow_user = GLS_Apr_weekend.x_

24x30 Array{Float64,2}:
 2145.86  1838.03  1787.53  2499.06  …  1532.97  1822.93   2089.49  1470.89
 1919.36  2234.54  2530.34  2967.14     2228.66  1716.73   2007.27  2755.1 
 2625.05  2398.24  2088.94  2426.37     1979.68  2240.81   2118.98  1888.16
 2539.48  2815.03  2926.87  3300.89     1721.72  1830.36   2517.52  2949.26
 2565.94  2651.87  2395.74  2319.8      2226.21  2122.4    2166.88  2287.79
 2526.44  3165.0   5082.23  3519.52  …  2374.31  1753.98   2338.65  5535.94
 2297.39  2286.0   2031.24  2079.66     1862.24  1653.94   1940.96  2468.52
 1905.67  2201.92  1877.32  3696.87     1730.31  1618.64   1778.8   3021.88
 2904.24  4531.49  4197.81  4413.11     4058.6   2301.68   2463.97  4386.34
 2909.25  6522.7   6384.04  6118.6      3656.59  1409.38   2430.28  5995.56
 2947.98  4268.55  5220.57  5235.21  …  3622.56  2607.33   1932.15  4855.45
 2802.79  4396.48  4398.06  4895.27     4414.85  2429.3    2443.83  4699.74
 1594.66  4776.13  5022.85  5140.21     1276.86  1172.54   1986.

In [19]:
function socialObj(linkFlowVec)
    objVal = sum([free_flow_time[a] * fcoeffs[1] * linkFlowVec[a] + free_flow_time[a] * fcoeffs[2] * linkFlowVec[a]^2 / capacity[a] + free_flow_time[a] * fcoeffs[3] * linkFlowVec[a]^3 / capacity[a]^2 + free_flow_time[a] * fcoeffs[4] * linkFlowVec[a]^4 / capacity[a]^3 + free_flow_time[a] * fcoeffs[5] * linkFlowVec[a]^5 / capacity[a]^4 + free_flow_time[a] * fcoeffs[6] * linkFlowVec[a]^6 / capacity[a]^5 + free_flow_time[a] * fcoeffs[7] * linkFlowVec[a]^7 / capacity[a]^6 + free_flow_time[a] * fcoeffs[8] * linkFlowVec[a]^8 / capacity[a]^7 + free_flow_time[a] * fcoeffs[9] * linkFlowVec[a]^9 / capacity[a]^8 for a = 1:numLinks])
    return objVal
end

socialObj (generic function with 1 method)

In [20]:
weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

for i in weekend_Apr_list
    println(socialObj(flow_user[:, i])/getObjectiveValue(m))
end

1.7276359082663204
1.4535870661096035
1.8831015515818645
1.3671288177916
1.7030046968611627
1.335203389723173
1.6732255147378796
1.3365776419086006
1.5468530754375809
